In [12]:
from math import exp
from random import seed
from random import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Auxilary classes and methods

In [13]:
def train_test_split(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_size]
    train_indices = shuffled_indices[test_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [14]:
def transform_labels(group_set):
    new_labels = np.zeros((len(group_set), group_set.nunique()))
    for i, label in enumerate(group_set):
            new_labels[i][label-1] = 1
    return new_labels

# Data importation

In [15]:
Y = pd.read_csv('digits_labels.csv', header=None)
X = pd.read_csv('digits_data.csv', header=None)
Y.columns = ['labels']
data = pd.concat([X, Y], axis = 1)

# Train-test split

In [16]:
# Ramdomly split the data into test and train, but keep digits balanced in each
k = 9
train_set = pd.DataFrame()
test_set = pd.DataFrame()
for i in range(k+1):
    data_temp = data[data['labels']==i+1]
    train_temp, test_temp = train_test_split(data_temp, 0.2)
    train_set = pd.concat([train_set, train_temp], axis = 0)
    test_set = pd.concat([test_set, test_temp], axis = 0)

In [17]:
X_train = train_set.iloc[:, :-1]
Y_train = train_set.iloc[:, -1]
X_test = test_set.iloc[:, :-1]
Y_test = test_set.iloc[:, -1]

In [18]:
Y_train_transformed = transform_labels(Y_train)
Y_test_transformed = transform_labels(Y_test)

n = X_train.shape[0] # sample size
d = X_train.shape[1] # number of features
k = Y_train.nunique() # number of classes

input_no = d
output_no = k
hidden_no = 100

In [19]:
X = X_train
y = Y_train_transformed

In [69]:
X.shape

(4000, 400)

In [70]:
y.shape

(4000, 10)

# Neural network classification

In [46]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis = 1).reshape(-1, 1)

def sigmoid_derivative(x):
    return np.exp(-x) / ((1 + np.exp(-x)) ** 2)

def forward_propagation(X, W1, W2, b1, b2):
    a1 = np.dot(X, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y_pred = softmax(a2)
    return a1, z1, a2, y_pred

def compute_cost(y, y_pred):
    J = 0.5 * sum((y - y_pred) ** 2)
    return J

def backward_propagation(X, y, y_pred, z2, a1, W2, z1):
    delta2 = np.multiply(-(y - y_pred), sigmoid_derivative(z2))
    dJ_dW2 = np.dot(a1.T, delta2)
    delta1 = np.dot(delta2, W2.T) * sigmoid_derivative(z1)
    dJ_dW1 = np.dot(X.T, delta1)
    return dJ_dW1, dJ_dW2

class NeuralNetwork():
    def __init__(self):
        self.W1 = None
        self.W2 = None
        self.b1 = None
        self.b2 = None
    
    def fit(self, X, y, alpha = 0.01, num_iterations = 1000, lambda_ = 0.01, input_no = 3, output_no = 2, hidden_no = 10):
        X = np.array(X)
        y = np.array(y)
        
        W1 = np.random.randn(input_no, hidden_no)
        b1 = np.ones([1, hidden_no])

        W2 = np.random.randn(hidden_no, output_no)
        b2 = np.ones([1, output_no])
        
        for i in range(num_iterations):
            z1, a1, z2, y_pred = forward_propagation(X, W1, W2, b1, b2)
            dJ_dW1, dJ_dW2 = backward_propagation(X, y, y_pred, z2, a1, W2, z1)

            W1 = W1 - (alpha * (dJ_dW1 + lambda_ * np.sign(W1))) #l1 penalty
            W2 = W2 - (alpha * (dJ_dW2 + lambda_ * np.sign(W2)))
            
        self.W1 = W1
        self.W2 = W2
        self.b1 = b1
        self.b2 = b2
        
        return self

    def predict(self, X):
        y_prob = forward_propagation(X, self.W1, self.W2, self.b1, self.b2)[3]
        max_prob_ind = np.argmax(y_prob, axis = 1)
        #y_class = np.array([labels[i] for i in max_prob_ind])
        
        return y_prob, max_prob_ind + 1

In [33]:
nn = NeuralNetwork().fit(X_train, Y_train_transformed, input_no = 400, output_no = 10)

In [34]:
nn.predict(X_train)

(array([[8.78231081e-01, 1.48937069e-06, 4.54465754e-05, ...,
         1.08577173e-01, 1.11719456e-02, 1.61628721e-05],
        [9.68120776e-01, 2.83606640e-05, 8.02695858e-05, ...,
         2.93637487e-02, 1.11467057e-03, 3.36585832e-05],
        [9.94780368e-01, 1.07559993e-04, 9.11154873e-04, ...,
         2.53621953e-03, 7.54656818e-04, 4.64018333e-06],
        ...,
        [2.88358577e-08, 7.71880086e-04, 1.67927315e-06, ...,
         3.17566187e-05, 1.34771701e-04, 9.96744123e-01],
        [3.41348606e-08, 8.48077362e-04, 2.08551389e-06, ...,
         3.66978801e-05, 1.50312089e-04, 9.96804986e-01],
        [3.97417391e-08, 1.30565179e-03, 3.66812934e-06, ...,
         1.25549171e-07, 1.40881978e-05, 9.90962918e-01]]),
 array([ 1,  1,  1, ..., 10, 10, 10], dtype=int64))

In [70]:
sum(nn.predict(X)[1] != np.array(Y_train))

164

# Tune l1 penalty strength by cross validation

In [47]:
import math

X = pd.DataFrame(X)
y = pd.DataFrame(y)

n_train = X.shape[0]
num_fold = 5
n_fold = math.floor(n_train / num_fold)

total_error_all = []

meshgrid = np.arange(0, 0.1, 0.01)

for lambda_ in meshgrid:
    total_error = 0
    for cur_fold in range(num_fold):
        X_fold = X.iloc[cur_fold * n_fold : (cur_fold + 1) * n_fold]
        y_fold = y.iloc[cur_fold * n_fold : (cur_fold + 1) * n_fold]
        X_train_fold = X.iloc[np.r_[0 : cur_fold * n_fold, (cur_fold + 1) * n_fold : n_train]]
        y_train_fold = y.iloc[np.r_[0 : cur_fold * n_fold, (cur_fold + 1) * n_fold : n_train]]

        pr = NeuralNetwork().fit(X_train_fold, y_train_fold, input_no = 400, output_no = 10, lambda_ = lambda_)
        y_pred = pr.predict(X_fold)[0]
        total_error += compute_cost(y_fold, y_pred)
    total_error_all.append(total_error)
    
best_lambda = meshgrid[np.argsort(total_error_all)[0]]
print(best_lambda)

0.0
